In [1]:
import lsdb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dask.distributed import Client

In [2]:
path = "/epyc/data3/hats/catalogs/dp1/dia_object_collection"

In [3]:
catalog = lsdb.open_catalog(path)
catalog

,dec,diaObjectId,nDiaSources,ra,radecMjdTai,tract,diaObjectForcedSource,diaSource
npartitions=208,,,,,,,,
"Order: 6, Pixel: 130",double[pyarrow],int64[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],"nested<band: [string], coord_dec: [double], co...","nested<band: [string], centroid_flag: [bool], ..."
"Order: 6, Pixel: 136",...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
"Order: 11, Pixel: 36833621",...,...,...,...,...,...,...,...
"Order: 7, Pixel: 143884",...,...,...,...,...,...,...,...


In [4]:
def count_uniq(df):
    uniq, count = np.unique(
        df['diaObjectForcedSource.forcedSourceOnDiaObjectId'],
        return_counts=True,
    )
    return pd.DataFrame({'uniq': uniq, 'count': count})

with Client(n_workers=8, threads_per_worker=1, dashboard_address=":18787") as client:
    df = catalog.map_partitions(count_uniq).compute()
df

/astro/users/kmalanch/.virtualenvs/default/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 18787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35165 instead
  warnings.warn(


,uniq,count
0,600386153916399617,1
1,600386153916399618,1
...,...,...
15,600438906849067022,1
16,600438906849067026,1


In [ ]:
duplicates = df.groupby('uniq').sum().query('count > 1')
duplicates.index.name = 'forcedSourceOnDiaObjectId'
duplicates.to_parquet('duplicated_forcedSourceOnDiaObjectId.parquet')
duplicates

In [ ]:
frac_of_duplicates = (
    duplicates['count'].sum()
    / catalog.map_partitions(lambda df: df.diaObjectForcedSource.nest.flat_length).compute().sum()
)
print(f"Fraction of duplicates: {frac_of_duplicates*100:.1f}%")

In [ ]:
with Client(n_workers=8, threads_per_worker=1, dashboard_address=":18787") as client:
    result = catalog.query("diaObjectForcedSource.forcedSourceOnDiaObjectId == 600386155389124609").dropna(subset=['diaObjectForcedSource']).compute()
result

In [ ]:
# Single pixel, with AT2014ahyy

import nested_pandas as npd

nf = npd.read_parquet(
    '/epyc/data3/hats/catalogs/dp1/dia_object_collection/dia_object_lc/dataset/Norder=9/Dir=2290000/Npix=2299828.parquet',
)

sources = nf['diaObjectForcedSource'].nest.to_flat()
mask = sources['forcedSourceOnDiaObjectId'].isin(duplicates.index)
plt.plot(sources[~mask].coord_ra, sources[~mask].coord_dec, 'o', ms=2, label='unique')
plt.plot(sources[mask].coord_ra, sources[mask].coord_dec, 'x', ms=2, label='duplicated')
plt.gca().invert_xaxis()
plt.xlabel('RA')
plt.ylabel('Dec')
plt.legend()